In [ ]:
import requests
import pandas as pd
import numpy as np
import datetime
import traceback
import datetime
from datetime import timedelta

# 데이터 분석 기본 라이브러리 관련
import os
import sys
import platform
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook

# EDA 관련
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import missingno as msno

# 데이터 전처리 관련
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
tf.random.set_seed(123)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/서영')

In [ ]:
삼성전자=pd.read_csv('./삼성전자.csv',encoding='cp949')
SK하이닉스=pd.read_csv('./SK하이닉스.csv',encoding='cp949')
NAVER=pd.read_csv('./NAVER.csv',encoding='cp949')
씨젠=pd.read_csv('./씨젠.csv',encoding='cp949')
우리들휴브레인=pd.read_csv('./우리들휴브레인.csv',encoding='cp949')
현대차=pd.read_csv('./현대차.csv',encoding='cp949')
DGB금융지주=pd.read_csv('./DGB금융지주.csv',encoding='cp949')
미스터블루=pd.read_csv('./미스터블루.csv',encoding='cp949')
셀트리온=pd.read_csv('./셀트리온.csv',encoding='cp949')
데일리블록체인=pd.read_csv('./데일리블록체인.csv',encoding='cp949')
소리바다=pd.read_csv('./소리바다.csv',encoding='cp949')
한화솔루션=pd.read_csv('./한화솔루션.csv',encoding='cp949')
아모레퍼시픽=pd.read_csv('./아모레퍼시픽.csv',encoding='cp949')
CJ대한통운=pd.read_csv('./CJ대한통운.csv',encoding='cp949')
GS건설=pd.read_csv('./GS건설.csv',encoding='cp949')

In [ ]:
stocks = ['삼성전자', 'SK하이닉스','NAVER','씨젠','우리들휴브레인','현대차','DGB금융지주','미스터블루',
          '셀트리온','데일리블록체인','소리바다','한화솔루션','아모레퍼시픽','CJ대한통운','GS건설']

In [ ]:
def model(df):
  data = df.sort_index(ascending=True, axis=0)
  new_data = pd.DataFrame(index=range(0,len(df)),columns=['날짜', '종가'])
  for i in range(0,len(data)):
      new_data['날짜'][i] = data['날짜'][i]
      new_data['종가'][i] = data['종가'][i]

  #setting index
  new_data.index = new_data.날짜
  new_data.drop('날짜', axis=1, inplace=True)

  final_test = df[df.날짜.str[:7].apply(lambda x: x in ['2019.09', '2019.10', '2019.11', '2019.12'])]
  test_start = final_test.index[0]
  test_last = final_test.index[-1]

  dataset = new_data.values
  train = dataset[0 : test_start]
  test = dataset[test_start : test_last+1]

  #converting dataset into x_train and y_train
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_data = scaler.fit_transform(dataset)

  x_train, y_train = [], []
  for i in range(20,len(train)):
      x_train.append(scaled_data[i-20:i,0])
      y_train.append(scaled_data[i,0])
  x_train, y_train = np.array(x_train), np.array(y_train)

  x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
  x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

  #predicting 246 values, using past 20 from the train data
  new = new_data[:test_last]
  inputs = new[len(new) - len(test) - 20:].values
  inputs  = scaler.transform(inputs)

  X_test = []
  for i in range(20,inputs.shape[0]):
      X_test.append(inputs[i-20:i,0])
  X_test = np.array(X_test)

  X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

  model = Sequential()
  model.add(LSTM(units=16, return_sequences=False, input_shape=(x_train.shape[1],1)))
  model.add(Dense(1))
  model_path=os.getcwd()
  model.compile(loss='mean_squared_error', optimizer='adam')
  early_stop = EarlyStopping(monitor='val_loss', patience=5)
  filename = os.path.join(model_path, 'tmp_checkpoint.h5')
  checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

  history = model.fit(x_train, y_train, 
                      epochs=200, 
                      batch_size=16,
                      validation_data=(x_valid, y_valid), 
                      callbacks=[early_stop, checkpoint])
  
  closing_price = model.predict(X_test)
  closing_price = scaler.inverse_transform(closing_price)

  return closing_price

In [ ]:
pred = pd.read_csv('./lstm.csv')

In [ ]:
삼성예측 = model(삼성전자)

In [ ]:
pred['삼성'] = np.array(삼성예측).ravel().tolist()
pred

,삼성
0,36948.218750
1,37364.648438
2,37234.984375
3,36783.667969
4,37408.453125
...,...
76,49464.195312
77,49473.144531
78,49072.144531
79,48617.332031


In [ ]:
closing_price  = model(SK하이닉스)

Epoch 1/200
235/242 [============================>.] - ETA: 0s - loss: 0.0019
Epoch 00001: val_loss improved from inf to 0.00023, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 10ms/step - loss: 0.0018 - val_loss: 2.2600e-04
Epoch 2/200
238/242 [============================>.] - ETA: 0s - loss: 2.0014e-04
Epoch 00002: val_loss improved from 0.00023 to 0.00021, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 8ms/step - loss: 2.0110e-04 - val_loss: 2.0997e-04
Epoch 3/200
241/242 [============================>.] - ETA: 0s - loss: 2.0524e-04
Epoch 00003: val_loss did not improve from 0.00021
242/242 [==============================] - 2s 7ms/step - loss: 2.0499e-04 - val_loss: 2.6904e-04
Epoch 4/200
238/242 [============================>.] - ETA: 0s - loss: 1.8720e-04
Epo

In [ ]:
pred['SK하이닉스'] = np.array(closing_price).ravel().tolist()

In [ ]:
closing_price = model(NAVER)
pred['NAVER'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
203/208 [============================>.] - ETA: 0s - loss: 0.0109
Epoch 00001: val_loss improved from inf to 0.00049, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
208/208 [==============================] - 2s 10ms/step - loss: 0.0107 - val_loss: 4.9062e-04
Epoch 2/200
201/208 [===========================>..] - ETA: 0s - loss: 9.4301e-04
Epoch 00002: val_loss did not improve from 0.00049
208/208 [==============================] - 1s 7ms/step - loss: 9.2684e-04 - val_loss: 5.3505e-04
Epoch 3/200
206/208 [============================>.] - ETA: 0s - loss: 8.4224e-04
Epoch 00003: val_loss improved from 0.00049 to 0.00044, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
208/208 [==============================] - 2s 8ms/step - loss: 8.3952e-04 - val_loss: 4.4487e-04
Epoch 4/200
206/208 [============================>.] - ETA: 0s - loss: 7.6653e-04
Epo

In [ ]:
closing_price = model(씨젠)
pred['씨젠'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
104/110 [===========================>..] - ETA: 0s - loss: 8.4982e-04
Epoch 00001: val_loss improved from inf to 0.00016, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
110/110 [==============================] - 2s 15ms/step - loss: 8.1281e-04 - val_loss: 1.5564e-04
Epoch 2/200
104/110 [===========================>..] - ETA: 0s - loss: 1.5301e-04
Epoch 00002: val_loss improved from 0.00016 to 0.00015, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
110/110 [==============================] - 1s 8ms/step - loss: 1.4950e-04 - val_loss: 1.4699e-04
Epoch 3/200
109/110 [============================>.] - ETA: 0s - loss: 1.4529e-04
Epoch 00003: val_loss did not improve from 0.00015
110/110 [==============================] - 1s 7ms/step - loss: 1.4465e-04 - val_loss: 1.5034e-04
Epoch 4/200
106/110 [===========================>..] - ETA: 0s - loss: 1.3046

In [ ]:
closing_price = model(우리들휴브레인)
pred['우리들휴브레인'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
118/119 [============================>.] - ETA: 0s - loss: 0.0107
Epoch 00001: val_loss improved from inf to 0.00141, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
119/119 [==============================] - 1s 13ms/step - loss: 0.0107 - val_loss: 0.0014
Epoch 2/200
114/119 [===========================>..] - ETA: 0s - loss: 0.0014
Epoch 00002: val_loss improved from 0.00141 to 0.00122, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
119/119 [==============================] - 1s 8ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 3/200
118/119 [============================>.] - ETA: 0s - loss: 0.0012
Epoch 00003: val_loss improved from 0.00122 to 0.00120, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
119/119 [==============================] - 1s 7ms/step - loss: 0.0012 - val_loss: 0.

In [ ]:
closing_price = model(현대차)
pred['현대차'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
238/242 [============================>.] - ETA: 0s - loss: 0.0139
Epoch 00001: val_loss improved from inf to 0.00056, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 10ms/step - loss: 0.0137 - val_loss: 5.5845e-04
Epoch 2/200
235/242 [============================>.] - ETA: 0s - loss: 5.5206e-04
Epoch 00002: val_loss improved from 0.00056 to 0.00052, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 7ms/step - loss: 5.5473e-04 - val_loss: 5.1739e-04
Epoch 3/200
235/242 [============================>.] - ETA: 0s - loss: 5.2506e-04
Epoch 00003: val_loss did not improve from 0.00052
242/242 [==============================] - 2s 7ms/step - loss: 5.2620e-04 - val_loss: 5.4293e-04
Epoch 4/200
236/242 [============================>.] - ETA: 0s - loss: 4.8853e-04
Epo

In [ ]:
closing_price = model(DGB금융지주)
pred['DGB금융지주'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
 94/101 [==========================>...] - ETA: 0s - loss: 0.0427
Epoch 00001: val_loss improved from inf to 0.00268, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
101/101 [==============================] - 1s 14ms/step - loss: 0.0402 - val_loss: 0.0027
Epoch 2/200
 94/101 [==========================>...] - ETA: 0s - loss: 0.0015
Epoch 00002: val_loss improved from 0.00268 to 0.00086, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
101/101 [==============================] - 1s 8ms/step - loss: 0.0014 - val_loss: 8.5614e-04
Epoch 3/200
 96/101 [===========================>..] - ETA: 0s - loss: 0.0010
Epoch 00003: val_loss improved from 0.00086 to 0.00082, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
101/101 [==============================] - 1s 7ms/step - loss: 0.0010 - val_loss: 8.1908e

In [ ]:
closing_price = model(미스터블루)
pred['미스터블루'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
54/57 [===========================>..] - ETA: 0s - loss: 0.0049
Epoch 00001: val_loss improved from inf to 0.00216, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
57/57 [==============================] - 1s 18ms/step - loss: 0.0047 - val_loss: 0.0022
Epoch 2/200
49/57 [========================>.....] - ETA: 0s - loss: 0.0014
Epoch 00002: val_loss improved from 0.00216 to 0.00122, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
57/57 [==============================] - 0s 8ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 3/200
50/57 [=========================>....] - ETA: 0s - loss: 0.0011
Epoch 00003: val_loss improved from 0.00122 to 0.00118, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
57/57 [==============================] - 0s 8ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 4

In [ ]:
closing_price = model(셀트리온)
pred['셀트리온'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
172/174 [============================>.] - ETA: 0s - loss: 0.0047
Epoch 00001: val_loss improved from inf to 0.00040, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
174/174 [==============================] - 2s 12ms/step - loss: 0.0047 - val_loss: 4.0379e-04
Epoch 2/200
168/174 [===========================>..] - ETA: 0s - loss: 4.7365e-04
Epoch 00002: val_loss improved from 0.00040 to 0.00039, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
174/174 [==============================] - 1s 7ms/step - loss: 4.7086e-04 - val_loss: 3.8776e-04
Epoch 3/200
172/174 [============================>.] - ETA: 0s - loss: 4.1897e-04
Epoch 00003: val_loss did not improve from 0.00039
174/174 [==============================] - 1s 7ms/step - loss: 4.1677e-04 - val_loss: 4.6596e-04
Epoch 4/200
173/174 [============================>.] - ETA: 0s - loss: 3.9444e-04
Epo

In [ ]:
closing_price = model(데일리블록체인)
pred['데일리블록체인'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
94/99 [===========================>..] - ETA: 0s - loss: 0.0116
Epoch 00001: val_loss improved from inf to 0.00201, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
99/99 [==============================] - 1s 14ms/step - loss: 0.0111 - val_loss: 0.0020
Epoch 2/200
98/99 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 00002: val_loss improved from 0.00201 to 0.00173, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
99/99 [==============================] - 1s 7ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 3/200
99/99 [==============================] - ETA: 0s - loss: 0.0015
Epoch 00003: val_loss improved from 0.00173 to 0.00165, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
99/99 [==============================] - 1s 8ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 4

In [ ]:
closing_price = model(소리바다)
pred['소리바다'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
219/220 [============================>.] - ETA: 0s - loss: 0.0050
Epoch 00001: val_loss improved from inf to 0.00072, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
220/220 [==============================] - 2s 11ms/step - loss: 0.0050 - val_loss: 7.1954e-04
Epoch 2/200
217/220 [============================>.] - ETA: 0s - loss: 5.2743e-04
Epoch 00002: val_loss improved from 0.00072 to 0.00057, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
220/220 [==============================] - 2s 7ms/step - loss: 5.3220e-04 - val_loss: 5.7300e-04
Epoch 3/200
213/220 [============================>.] - ETA: 0s - loss: 4.7559e-04
Epoch 00003: val_loss improved from 0.00057 to 0.00050, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
220/220 [==============================] - 2s 7ms/step - loss: 4

In [ ]:
closing_price = model(한화솔루션)
pred['한화솔루션'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
237/242 [============================>.] - ETA: 0s - loss: 0.0101
Epoch 00001: val_loss improved from inf to 0.00080, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 10ms/step - loss: 0.0099 - val_loss: 7.9922e-04
Epoch 2/200
237/242 [============================>.] - ETA: 0s - loss: 6.0323e-04
Epoch 00002: val_loss improved from 0.00080 to 0.00065, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 8ms/step - loss: 6.0292e-04 - val_loss: 6.4549e-04
Epoch 3/200
239/242 [============================>.] - ETA: 0s - loss: 5.3880e-04
Epoch 00003: val_loss improved from 0.00065 to 0.00061, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 8ms/step - loss: 5

In [ ]:
closing_price = model(아모레퍼시픽)
pred['아모레퍼시픽'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
159/162 [============================>.] - ETA: 0s - loss: 0.0093
Epoch 00001: val_loss improved from inf to 0.00153, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
162/162 [==============================] - 2s 11ms/step - loss: 0.0091 - val_loss: 0.0015
Epoch 2/200
159/162 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 00002: val_loss improved from 0.00153 to 0.00128, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
162/162 [==============================] - 1s 7ms/step - loss: 0.0016 - val_loss: 0.0013
Epoch 3/200
162/162 [==============================] - ETA: 0s - loss: 0.0014
Epoch 00003: val_loss improved from 0.00128 to 0.00114, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
162/162 [==============================] - 1s 8ms/step - loss: 0.0014 - val_loss: 0.

In [ ]:
closing_price = model(CJ대한통운)
pred['CJ대한통운'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
235/242 [============================>.] - ETA: 0s - loss: 0.0285
Epoch 00001: val_loss improved from inf to 0.00080, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 10ms/step - loss: 0.0278 - val_loss: 7.9660e-04
Epoch 2/200
237/242 [============================>.] - ETA: 0s - loss: 7.5854e-04
Epoch 00002: val_loss improved from 0.00080 to 0.00076, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 7ms/step - loss: 7.5486e-04 - val_loss: 7.6277e-04
Epoch 3/200
237/242 [============================>.] - ETA: 0s - loss: 6.7381e-04
Epoch 00003: val_loss improved from 0.00076 to 0.00062, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 8ms/step - loss: 6

In [ ]:
closing_price = model(GS건설)
pred['GS건설'] = np.array(closing_price).ravel().tolist()

Epoch 1/200
236/242 [============================>.] - ETA: 0s - loss: 0.0138
Epoch 00001: val_loss improved from inf to 0.00075, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 10ms/step - loss: 0.0135 - val_loss: 7.5090e-04
Epoch 2/200
240/242 [============================>.] - ETA: 0s - loss: 7.3667e-04
Epoch 00002: val_loss improved from 0.00075 to 0.00061, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 7ms/step - loss: 7.3535e-04 - val_loss: 6.1178e-04
Epoch 3/200
241/242 [============================>.] - ETA: 0s - loss: 6.0997e-04
Epoch 00003: val_loss improved from 0.00061 to 0.00061, saving model to /content/drive/.shortcut-targets-by-id/1t6-0BUco9HRnHygsMqUj5ICBmose6jur/서영/tmp_checkpoint.h5
242/242 [==============================] - 2s 7ms/step - loss: 6

In [ ]:
pred

,Unnamed: 0,삼성,SK하이닉스,NAVER,씨젠,우리들휴브레인,현대차,DGB금융지주,미스터블루,셀트리온,데일리블록체인,소리바다,한화솔루션,아모레퍼시픽,CJ대한통운,GS건설
0,0,36948.218750,73416.648438,143216.406250,21505.148438,2150.327148,126491.046875,6836.139648,9285.294922,156928.843750,671.614868,868.573486,16592.710938,122328.570312,140238.218750,30960.367188
1,1,37364.648438,75685.351562,143754.656250,22022.261719,2184.906738,128012.726562,6875.688477,9335.557617,157671.953125,703.280701,867.475037,17084.638672,124881.296875,142604.750000,31613.724609
2,2,37234.984375,77010.281250,145993.671875,22597.400391,2169.770752,130304.789062,6898.837402,9417.095703,157103.437500,714.883423,874.833984,17024.187500,127300.968750,145506.343750,31374.287109
3,3,36783.667969,77296.820312,147270.593750,22378.529297,2194.916016,128644.250000,6926.433594,9917.833984,159793.812500,719.636719,897.546570,17153.869141,132606.640625,142781.812500,31324.634766
4,4,37408.453125,78901.351562,148102.984375,22438.500000,2189.519531,128341.484375,7004.859863,10025.784180,166010.265625,730.143860,907.086731,17210.400391,135352.046875,142315.031250,31200.482422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76,49464.195312,92590.414062,176118.562500,29737.851562,3365.480957,123216.843750,7516.757324,8068.590332,180353.953125,1062.318604,795.936707,18467.898438,194484.562500,151921.671875,30793.166016
77,77,49473.144531,93562.406250,178821.000000,29987.201172,3137.034180,123721.453125,7509.876953,8090.340332,181956.343750,1016.150391,787.832458,18987.160156,196328.468750,153207.781250,31086.660156
78,78,49072.144531,93726.273438,180231.640625,30227.541016,3208.735107,122588.210938,7510.377441,7822.808105,185781.859375,997.894897,784.469727,18877.746094,195598.609375,151904.781250,31266.740234
79,79,48617.332031,93301.085938,179584.875000,30726.134766,3143.763672,121986.257812,7499.477539,7346.560547,185496.296875,974.309021,769.678772,18684.798828,195422.687500,150311.796875,31115.775391


In [ ]:
pred.drop(['Unnamed: 0'], axis='columns', inplace=True)

In [ ]:
pred

,삼성,SK하이닉스,NAVER,씨젠,우리들휴브레인,현대차,DGB금융지주,미스터블루,셀트리온,데일리블록체인,소리바다,한화솔루션,아모레퍼시픽,CJ대한통운,GS건설
0,36948.218750,73416.648438,143216.406250,21505.148438,2150.327148,126491.046875,6836.139648,9285.294922,156928.843750,671.614868,868.573486,16592.710938,122328.570312,140238.218750,30960.367188
1,37364.648438,75685.351562,143754.656250,22022.261719,2184.906738,128012.726562,6875.688477,9335.557617,157671.953125,703.280701,867.475037,17084.638672,124881.296875,142604.750000,31613.724609
2,37234.984375,77010.281250,145993.671875,22597.400391,2169.770752,130304.789062,6898.837402,9417.095703,157103.437500,714.883423,874.833984,17024.187500,127300.968750,145506.343750,31374.287109
3,36783.667969,77296.820312,147270.593750,22378.529297,2194.916016,128644.250000,6926.433594,9917.833984,159793.812500,719.636719,897.546570,17153.869141,132606.640625,142781.812500,31324.634766
4,37408.453125,78901.351562,148102.984375,22438.500000,2189.519531,128341.484375,7004.859863,10025.784180,166010.265625,730.143860,907.086731,17210.400391,135352.046875,142315.031250,31200.482422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,49464.195312,92590.414062,176118.562500,29737.851562,3365.480957,123216.843750,7516.757324,8068.590332,180353.953125,1062.318604,795.936707,18467.898438,194484.562500,151921.671875,30793.166016
77,49473.144531,93562.406250,178821.000000,29987.201172,3137.034180,123721.453125,7509.876953,8090.340332,181956.343750,1016.150391,787.832458,18987.160156,196328.468750,153207.781250,31086.660156
78,49072.144531,93726.273438,180231.640625,30227.541016,3208.735107,122588.210938,7510.377441,7822.808105,185781.859375,997.894897,784.469727,18877.746094,195598.609375,151904.781250,31266.740234
79,48617.332031,93301.085938,179584.875000,30726.134766,3143.763672,121986.257812,7499.477539,7346.560547,185496.296875,974.309021,769.678772,18684.798828,195422.687500,150311.796875,31115.775391


In [ ]:
pred.to_csv('./lstm.csv')